In [2]:
# train3에서 minmax scaling 진행


In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

In [15]:
import warnings

# 경고 무시
warnings.filterwarnings("ignore", category=FutureWarning)

In [16]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [17]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
train = train.drop(['user_id'], axis=1)
test_id = test['user_id'].values
test = test.drop(['user_id'], axis=1)



In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  object 
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [19]:
# preferred_difficulty_level 열 인코딩
train.loc[train['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
train.loc[train['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
train.loc[train['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2

test.loc[test['preferred_difficulty_level'].values == 'Low','preferred_difficulty_level'] = 0
test.loc[test['preferred_difficulty_level'].values == 'Medium','preferred_difficulty_level'] = 1
test.loc[test['preferred_difficulty_level'].values == 'High','preferred_difficulty_level'] = 2
train['preferred_difficulty_level'] = train['preferred_difficulty_level'].astype(int)

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  object 
 11  customer_inquiry_history           10000 non-null  int6

In [21]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 데이터 스케일링을 위한 StandardScaler 인스턴스 생성
scaler = StandardScaler()
categoric_col =['subscription_type']
label_encoders = {}
for col in categoric_col:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].fit_transform(test[col])

In [22]:
# 'payment_pattern' 열에 대한 고유값 확인
unique_values = train['payment_pattern'].unique()

# 각 고유값에 대해 새로운 열 생성 및 0 또는 1로 설정
for value in unique_values:
    train[f'payment_pattern_{value}'] = (train['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
train.drop('payment_pattern', axis=1, inplace=True)

for value in unique_values:
    test[f'payment_pattern_{value}'] = (test['payment_pattern'] == value).astype(int)

# 기존 'payment_pattern' 열 삭제
test.drop('payment_pattern', axis=1, inplace=True)

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   subscription_duration              10000 non-null  int64  
 1   recent_login_time                  10000 non-null  int64  
 2   average_login_time                 10000 non-null  float64
 3   average_time_per_learning_session  10000 non-null  float64
 4   monthly_active_learning_days       10000 non-null  int64  
 5   total_completed_courses            10000 non-null  int64  
 6   recent_learning_achievement        10000 non-null  float64
 7   abandoned_learning_sessions        10000 non-null  int64  
 8   community_engagement_level         10000 non-null  int64  
 9   preferred_difficulty_level         10000 non-null  int32  
 10  subscription_type                  10000 non-null  int32  
 11  customer_inquiry_history           10000 non-null  int6

In [24]:
# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# 선택한 열을 Min-Max Scaling
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
test[columns_to_scale] = scaler.fit_transform(test[columns_to_scale])

In [25]:
train['target'] = train['target'].astype('category')

In [26]:
X = train[train.columns.drop(['target'])]
Y = train['target']

In [27]:
# X와 Y로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=40)

In [28]:
# RandomOverSampler를 사용하여 오버샘플링 수행
ros = RandomOverSampler(random_state=42)
X_train, y_train = ros.fit_resample(X_train, y_train)

# randomForest



In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# RandomForest 모델 생성 및 학습
model_rf = RandomForestClassifier(random_state=42)

RandomForestClassifier(random_state=42)

In [170]:
# 탐색할 매개변수 그리드 정의
param_grid_RF = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    # 다른 매개변수들도 필요에 따라 추가할 수 있습니다.
}

In [172]:
# GridSearchCV를 사용하여 그리드 서치 수행
grid_search_RF = GridSearchCV(model_rf, param_grid_RF, cv=5, scoring='f1_macro')
grid_search_RF.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='f1_macro')

In [173]:
# 최적의 매개변수 출력
print("최적의 매개변수:", grid_search_RF.best_params_)

최적의 매개변수: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [174]:
# 최적의 모델에 대한 평가
best_model_RF = grid_search_RF.best_estimator_
y_pred_train_RF = best_model_RF.predict(X_test)
macro_f1_train_RF = f1_score(y_test, y_pred_train_RF, average='macro')
print(f'학습 세트 매크로 F1-score: {macro_f1_train_RF:.4f}')

학습 세트 매크로 F1-score: 0.4752


# xgboost 

In [33]:
import xgboost as xgb

# 데이터셋 로드 및 전처리
# 예: X는 특징 변수, y는 범주형 목표 변수


# XGBoost 모델 생성 및 학습
model_xgb = xgb.XGBClassifier(random_state=2200, objective="binary:logistic")
                              

param_grid = {
    'n_estimators': [200,230,250,300],
    'learning_rate': [None,0.1,0.2],
    'max_depth': [10],
    'reg_lambda': [None,0.1, 0.5, 0.7]
    # 다른 매개변수들도 필요에 따라 추가할 수 있습니다.
}

In [34]:
# GridSearchCV를 사용하여 그리드 서치 수행
grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=2200, ...),
             param_grid={'learning_rate': [None, 0.1, 0.2], 'max_depth': [10],
                         'n_estimators': [200, 230, 250, 300],
                         'reg_lambda': [None, 0.1, 0.5, 0.7]},
             scoring='f1_macro')

In [35]:
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df[["params", "mean_test_score", "rank_test_score", "split0_test_score", "split1_test_score", "split2_test_score"]]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.716077,13,0.661787,0.660311,0.674112
1,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.713829,25,0.663235,0.644969,0.666530
2,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.715849,15,0.670808,0.655041,0.682314
3,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.708490,48,0.658277,0.634484,0.669974
4,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.717441,6,0.657260,0.663947,0.671195
5,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.711881,31,0.665641,0.641022,0.667597
6,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.718232,3,0.673767,0.659528,0.685125
7,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.710839,38,0.658772,0.638482,0.669277
8,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.715937,14,0.658782,0.658394,0.672668
9,"{'learning_rate': None, 'max_depth': 10, 'n_es...",0.713467,27,0.667699,0.644969,0.670076


In [770]:
# 최적의 매개변수 출력
print("최적의 매개변수:", grid_search.best_params_)

최적의 매개변수: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 300, 'reg_alpha': 0.5, 'reg_lambda': 0.7}


In [771]:
# 최적의 모델에 대한 평가
best_model_xgb = grid_search.best_estimator_
y_pred_train_xgb = best_model_xgb.predict(X_test)
macro_f1_train_xgb = f1_score(y_test, y_pred_train_xgb, average='macro')
print(f'학습 세트 매크로 F1-score: {macro_f1_train_xgb:.4f}')

학습 세트 매크로 F1-score: 0.5053


# Catboost

In [177]:
from catboost import CatBoostClassifier


# CatBoost 모델 생성 및 학습
model_catboost = CatBoostClassifier(loss_function='MultiClass', cat_features=[0, 1])

In [178]:
# 탐색할 매개변수 그리드 정의
param_grid_catboost = {
    'iterations': [50, 100, 150],
    'depth': [3, 5, 10],
    'learning_rate': [0.1, 0.01],
    # 다른 매개변수들도 필요에 따라 추가할 수 있습니다.
}

In [183]:
# GridSearchCV를 사용하여 그리드 서치 수행
grid_search_catboost = GridSearchCV(model_catboost, param_grid_catboost, cv=5, scoring='f1_macro')
grid_search_catboost.fit(X_train, y_train)

0:	learn: 0.6899611	total: 25.6ms	remaining: 1.25s
1:	learn: 0.6872706	total: 49.2ms	remaining: 1.18s
2:	learn: 0.6850512	total: 75.2ms	remaining: 1.18s
3:	learn: 0.6830659	total: 101ms	remaining: 1.16s
4:	learn: 0.6816999	total: 144ms	remaining: 1.29s
5:	learn: 0.6801994	total: 161ms	remaining: 1.18s
6:	learn: 0.6788346	total: 187ms	remaining: 1.15s
7:	learn: 0.6776917	total: 222ms	remaining: 1.16s
8:	learn: 0.6767061	total: 252ms	remaining: 1.15s
9:	learn: 0.6758221	total: 289ms	remaining: 1.16s
10:	learn: 0.6748835	total: 318ms	remaining: 1.13s
11:	learn: 0.6740802	total: 343ms	remaining: 1.09s
12:	learn: 0.6731956	total: 366ms	remaining: 1.04s
13:	learn: 0.6726262	total: 391ms	remaining: 1s
14:	learn: 0.6723419	total: 418ms	remaining: 975ms
15:	learn: 0.6719837	total: 439ms	remaining: 933ms
16:	learn: 0.6719651	total: 454ms	remaining: 882ms
17:	learn: 0.6715789	total: 485ms	remaining: 863ms
18:	learn: 0.6713574	total: 512ms	remaining: 836ms
19:	learn: 0.6712099	total: 551ms	remaini

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x000002AC17D080D0>,
             param_grid={'depth': [3, 5, 10], 'iterations': [50, 100, 150],
                         'learning_rate': [0.1, 0.01]},
             scoring='f1_macro')

In [186]:
# 최적의 매개변수 출력
print("최적의 매개변수:", grid_search_catboost.best_params_)

최적의 매개변수: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200}


In [188]:
# 최적의 모델에 대한 평가
best_model_catboost= grid_search_catboost.best_estimator_
y_pred_train_catboost = best_model_catboost.predict(X_test)
macro_f1_train_catboost = f1_score(y_test, y_pred_train_catboost, average='macro')
print(f'학습 세트 매크로 F1-score: {macro_f1_train_catboost:.4f}')

학습 세트 매크로 F1-score: 0.4995


# decisiontreeclassifier

In [312]:
from sklearn.tree import DecisionTreeClassifier

model_DTC = DecisionTreeClassifier(random_state=1004)

model_DTC.fit(X_train, y_train)

DecisionTreeClassifier(random_state=1004)

In [313]:
y_pred_train_DTC = model_DTC.predict(X_test)

In [314]:
print(f1_score(y_test, y_pred_train_DTC, average='macro'))

0.5028382487647572


In [44]:
pred_final = best_model_xgb.predict(test)

NameError: name 'best_model_xgb' is not defined

In [45]:
submit = pd.DataFrame({
    'user_id': test_id,
    'target': pred_final.flatten()
})

NameError: name 'pred_final' is not defined

In [291]:
submit.to_csv("../submission_data/5.csv", index = False)

In [46]:
pred_final = model_xgb.predict(test)

In [47]:
submit = pd.DataFrame({
    'user_id': test_id,
    'target': pred_final.flatten()
})

In [48]:
submit.to_csv("../submission_data/5.csv", index = False)

In [53]:
import xgboost as xgb

# XGBoost 모델 생성 및 학습
model_xgb = xgb.XGBClassifier(random_state=2200,objective="binary:logistic",n_estimators= 200,reg_alpha = 0.1,reg_lambda = 0.1)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=2200, ...)

In [54]:
y_pred_xgb = model_xgb.predict(X_test)

In [55]:
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb}")

XGBoost Accuracy: 0.5265


In [52]:
print(f1_score(y_test,y_pred_xgb , average='macro'))

0.5086593309212308
